In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [4]:
from scr.analysis.perlayer import LayerLoss

In [6]:
layer_analysis_dict = LayerLoss(
    input_path="results/train_val_test",
    output_path="results/analysis_layer",
).layer_analysis_dict

In [5]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scalex-scaley-noloader",
    output_path="results/sklearn-scalex-scaley-noloader_layer",
).layer_analysis_dict

In [4]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader",
    output_path="results/sklearn-scaley-noloader_layer",
).layer_analysis_dict

In [10]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader-rand",
    output_path="results/sklearn-scaley-noloader-rand_layer",
).layer_analysis_dict

Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1b_t33_650M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1b_t33_650M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t6_43M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t6_43M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t12_85M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t12_85M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t34_670M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t34_670M_UR50S/mean ...


In [6]:
from scr.params.emb import TRANSFORMER_INFO

In [9]:
[""] + list(TRANSFORMER_INFO.keys())

['',
 'esm1_t6_43M_UR50S',
 'esm1_t12_85M_UR50S',
 'esm1_t34_670M_UR50S',
 'esm1b_t33_650M_UR50S']

In [8]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-fixa-noloader",
    output_path="results/sklearn-scaley-fixa-noloader_layer",
).layer_analysis_dict

In [6]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-fixa",
    output_path="results/sklearn-scaley-fixa_layer",
).layer_analysis_dict

In [32]:
layer_analysis_dict.keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"].keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"]["esm1b_t33_650M_UR50S"].keys()

(dict_keys(['proeng_gb1_two_vs_rest_mean', 'proeng_gb1_low_vs_high_mean', 'proeng_aav_one_vs_many_mean']),
 dict_keys(['esm1b_t33_650M_UR50S', 'esm1_t6_43M_UR50S', 'esm1_t12_85M_UR50S', 'esm1_t34_670M_UR50S']),
 dict_keys(['train_mse', 'test_ndcg', 'test_rho']))

In [26]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader",
    output_path="results/sklearn-scaley-noloader_layer",
).layer_analysis_dict

Making results/sklearn-scaley-noloader_layer-onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1 ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-oneh